In [11]:
import pandas as pd
first_chunk = True  # To ensure header is only written once
for chunk in pd.read_csv("/Users/joeyrich/Desktop/local/varseek/trash/testing.csv", chunksize=1):
    chunk.to_csv("/Users/joeyrich/Desktop/local/varseek/trash/testing2.csv", mode="w" if first_chunk else "a", index=False, header=first_chunk)
    first_chunk = False  # Ensure header is only written once

In [1]:
import os
import subprocess
import shutil
import sys
import pandas as pd
from varseek.utils import create_identity_t2g

varseek_directory = os.path.dirname(os.path.abspath(""))

conda_env_path = os.path.dirname(os.path.dirname(shutil.which("kb")))  # to get kb path
operating_system = "linux" if sys.platform.startswith("linux") else "darwin/m1"

kallisto = f"{conda_env_path}/lib/python3.10/site-packages/kb_python/bins/{operating_system}/kallisto/kallisto"  # or kallisto_k64
bustools = f"{conda_env_path}/lib/python3.10/site-packages/kb_python/bins/{operating_system}/bustools/bustools"

In [2]:
ref_fa = f"{varseek_directory}/tests/kb_files/ref_test.fa"
read1_fq = f"{varseek_directory}/tests/kb_files/reads1_test.fq"
read2_fq = f"{varseek_directory}/tests/kb_files/reads2_test.fq"
test_index = f"{varseek_directory}/tests/kb_files/index_test.idx"
test_t2g = f"{varseek_directory}/tests/kb_files/t2g_test.txt"
kb_count_out_test = f"{varseek_directory}/tests/kb_files/test_kb_count_out_March2025_with_num"
kb_count_out_test2 = f"{varseek_directory}/tests/kb_files/test_kb_count_out_March2025_without_num"

rnaseq_fastq_files_final = [read1_fq, read2_fq]
os.makedirs(kb_count_out_test, exist_ok=True)

if not os.path.exists(test_index):
    kb_ref_command = ["kb", "ref", "--workflow", "custom", "-t", str(8), "-i", test_index, "--d-list", "None", ref_fa]
    subprocess.run(kb_ref_command, check=True)

if not os.path.exists(test_t2g):
    create_identity_t2g(ref_fa, test_t2g)

if not os.path.exists(kb_count_out_test) or len(os.listdir(kb_count_out_test)) == 0:
    kb_count_command = ["kb", "count", "-t", str(2), "-i", test_index, "-g", test_t2g, "-x", "bulk", "--num", "--h5ad", "--parity", "single", "-o", kb_count_out_test, "--kallisto", kallisto, "--bustools", bustools] + rnaseq_fastq_files_final
    subprocess.run(kb_count_command, check=True)

if not os.path.exists(kb_count_out_test2) or len(os.listdir(kb_count_out_test2)) == 0:
    kb_count_command = ["kb", "count", "-t", str(2), "-i", test_index, "-g", test_t2g, "-x", "bulk", "--h5ad", "--parity", "single", "-o", kb_count_out_test2, "--kallisto", kallisto, "--bustools", bustools] + rnaseq_fastq_files_final
    subprocess.run(kb_count_command, check=True)

# kb_count_out_test = f"{varseek_directory}/tests/kb_files/test_kb_count_out_paired"
# if not os.path.exists(kb_count_out_test) or len(os.listdir(kb_count_out_test)) == 0:
#     kb_count_command = ["kb", "count", "-t", str(8), "-i", test_index, "-g", test_t2g, "-x", "bulk", "--num", "--h5ad", "--parity", "paired", "-o", kb_count_out_test] + rnaseq_fastq_files_final
#     subprocess.run(kb_count_command, check=True)

In [ ]:
bus_text_file = os.path.join(kb_count_out_test, "bus_text.txt")
bus_file = os.path.join(kb_count_out_test, "output.bus")
bustools_text_command = f"{bustools} text -o {bus_text_file} -f {bus_file}"
subprocess.run(bustools_text_command, check=True, shell=True)

bus_text_file = os.path.join(kb_count_out_test2, "bus_text.txt")
bus_file = os.path.join(kb_count_out_test2, "output.bus")
bustools_text_command = f"{bustools} text -o {bus_text_file} -f {bus_file}"
subprocess.run(bustools_text_command, check=True, shell=True)

In [ ]:
import importlib
import varseek.utils.seq_utils
importlib.reload(varseek.utils.varseek_clean_utils)
from varseek.utils.varseek_clean_utils import make_bus_df
bus_df = make_bus_df(kallisto_out = kb_count_out_test, fastq_file_list = rnaseq_fastq_files_final, t2g_file = test_t2g, mm = False, union = False, technology = "bulk", bustools = bustools)

In [5]:
read1_sample2_fq = f"{varseek_directory}/tests/kb_files/reads1_sample2_test.fq"
read2_sample2_fq = f"{varseek_directory}/tests/kb_files/reads2_sample2_test.fq"
rnaseq_fastq_files_final = [read1_fq, read2_fq, read1_sample2_fq, read2_sample2_fq]

kb_count_out_test_4_total_fastqs = f"{varseek_directory}/tests/kb_files/test_kb_count_out_March2025_with_num_4_total_fastqs"

if not os.path.exists(kb_count_out_test_4_total_fastqs) or len(os.listdir(kb_count_out_test_4_total_fastqs)) == 0:
    kb_count_command = ["kb", "count", "-t", str(2), "-i", test_index, "-g", test_t2g, "-x", "bulk", "--num", "--h5ad", "--parity", "single", "-o", kb_count_out_test_4_total_fastqs, "--kallisto", kallisto, "--bustools", bustools] + rnaseq_fastq_files_final
    subprocess.run(kb_count_command, check=True)

bus_text_file = os.path.join(kb_count_out_test_4_total_fastqs, "bus_text.txt")
bus_file = os.path.join(kb_count_out_test_4_total_fastqs, "output.bus")
bustools_text_command = f"{bustools} text -o {bus_text_file} -f {bus_file}"
subprocess.run(bustools_text_command, check=True, shell=True)

import importlib
import varseek.utils.seq_utils
importlib.reload(varseek.utils.varseek_clean_utils)
from varseek.utils.varseek_clean_utils import make_bus_df
bus_df = make_bus_df(kallisto_out = kb_count_out_test_4_total_fastqs, fastq_file_list = rnaseq_fastq_files_final, t2g_file = test_t2g, mm = False, union = False, technology = "bulk", bustools = bustools)

loading in transcripts
loading in barcodes
loading in fastq headers
loading in fastq headers
loading in fastq headers
loading in fastq headers
loading in ec matrix
loading in t2g df
running bustools text
loading in bus df
merging ec df into bus df
Apply the mapping function to create gene name columns
added counted in matrix column
saving bus df


Read in 12 BUS records


In [3]:
read1_sample2_fq = f"{varseek_directory}/tests/kb_files/reads1_sample2_test.fq"
read2_sample2_fq = f"{varseek_directory}/tests/kb_files/reads2_sample2_test.fq"
rnaseq_fastq_files_final = [read1_fq, read2_fq, read1_sample2_fq, read2_sample2_fq]

kb_count_out_test_4_total_fastqs_paired = f"{varseek_directory}/tests/kb_files/test_kb_count_out_March2025_with_num_4_total_fastqs_paired2"

# if not os.path.exists(kb_count_out_test_4_total_fastqs_paired) or len(os.listdir(kb_count_out_test_4_total_fastqs_paired)) == 0:
#     kb_count_command = ["kb", "count", "-t", str(2), "-i", test_index, "-g", test_t2g, "-x", "bulk", "--num", "--h5ad", "--parity", "paired", "-o", kb_count_out_test_4_total_fastqs_paired, "--kallisto", kallisto, "--bustools", bustools] + rnaseq_fastq_files_final
#     subprocess.run(kb_count_command, check=True)

# bus_text_file = os.path.join(kb_count_out_test_4_total_fastqs_paired, "bus_text.txt")
# bus_file = os.path.join(kb_count_out_test_4_total_fastqs_paired, "output.bus")
# bustools_text_command = f"{bustools} text -o {bus_text_file} -f {bus_file}"
# subprocess.run(bustools_text_command, check=True, shell=True)

# import importlib
# import varseek.utils.seq_utils
# importlib.reload(varseek.utils.varseek_clean_utils)
# from varseek.utils.varseek_clean_utils import make_bus_df

In [18]:
import importlib
import varseek.utils.seq_utils
importlib.reload(varseek.utils.varseek_clean_utils)
from varseek.utils.varseek_clean_utils import make_bus_df
df = make_bus_df(kb_count_out=kb_count_out_test_4_total_fastqs_paired, fastq_file_list=rnaseq_fastq_files_final, t2g_file=test_t2g, mm=False, union=False, technology="bulk", parity="paired", bustools=bustools, check_only=True)

File /Users/joeyrich/Desktop/local/varseek/tests/kb_files/reads1_test.fq does not match the expected bulk file naming convention of SAMPLE_PAIR.EXT where SAMPLE is sample name, PAIR is 1/2, and EXT is a fastq extension - or the Illumina file naming convention of SAMPLE_LANE_R[12]_001.fastq.gz, where SAMPLE is letters, numbers, underscores; LANE is numbers with optional leading 0s; pair is either R1 or R2; and it has .fq or .fastq extension (or .fq.gz or .fastq.gz).
File /Users/joeyrich/Desktop/local/varseek/tests/kb_files/reads2_test.fq does not match the expected bulk file naming convention of SAMPLE_PAIR.EXT where SAMPLE is sample name, PAIR is 1/2, and EXT is a fastq extension - or the Illumina file naming convention of SAMPLE_LANE_R[12]_001.fastq.gz, where SAMPLE is letters, numbers, underscores; LANE is numbers with optional leading 0s; pair is either R1 or R2; and it has .fq or .fastq extension (or .fq.gz or .fastq.gz).
File /Users/joeyrich/Desktop/local/varseek/tests/kb_files/re

Processing FASTQ headers: 3it [00:00, 2437.60it/s]
Processing FASTQ headers: 3it [00:00, 24151.46it/s]
Processing FASTQ headers: 3it [00:00, 7672.51it/s]
Processing FASTQ headers: 3it [00:00, 31223.11it/s]


loading in ec matrix
loading in t2g df
loading in bus df
Merging fastq header df and ec_df into bus df
Apply the mapping function to create gene name columns


100%|██████████| 4/4 [00:00<00:00, 7906.32it/s]


Taking set of gene_names


100%|██████████| 4/4 [00:00<00:00, 8062.09it/s]


Determining what counts in count matrix


100%|██████████| 4/4 [00:00<00:00, 4450.19it/s]

Saving bus df as parquet


In [ ]:
import varseek as vk
vk.ref(
    variants="cosmic",
    sequences="cdna",
    w=11
)

In [2]:
len("ENST00000397121:c.1099_1100insCTTTCAACTCGTCC")

44